In [3]:
__author__ = 'Brian Merino <brian.merino@noirlab.edu>'
__version__ = '2024' # yyyymmdd; version datestamp of this notebook
__keywords__ = ['Aladin','smash','des','delve']

# Displaying MOCs and HiPs in Aladin Lite v3

## Table of contents
* [Goals](#goals)
* [Summary](#summary)
* [Disclaimers and attribution](#disclaimer)
* [Imports and setup](#imports)
* [Create MOCs](#Create_MOCs)
* [About the dataset](#About)
* [Downloading data for reduction](#Downloading_Data)
* [Set up the DRAGONS logger](#DRAGONS_logger)
* [Create File Lists](#File_Lists)
* [Create Master Dark](#Master_dark)
* [Create Bad Pixel Mask](#BPM)
* [Create Master Flat Field](#Master_Flat)
* [Reduce Science Images](#Reduce_Science)
* [Display stacked final image](#Display_Image)
* [Clean-up (optional)](#Clean-up)

<a class="anchor" id="goals"></a>
# Goals
Showcase how to create Multi-Order Coverage maps (MOCs) and Hierarchical Progressive Surveys (HiPS) and display them on the Aladin Lite viewer.

The steps include downloading data from the Astro DataLab's data explorer, turning the data into MOCs and HiPs files, establish a local instance of the Aladin Lite viewer, and finally, display the files and explorer Aladin.


<a class="anchor" id="summary"></a>
# Summary
Aladin Lite is an interactable sky atlas that runs in your browser. 

Aladin makes it possible to explore the sky and has built in functionality that makes it possible to overlay images onto the map and identify objects included in databases. 

HiPS catalogs make it possible to see the locations of objects on the sky while MOCs display the footprints of surveys. MOCs make it possible to visualize overlaps in various surveys.

This notebook will demonstrate how to create and overlay several MOC files onto Aladin which would help identify overlapping datasets. It could also assist in preparing for a telescope proposal as you would be able to see whether or not any of the datasets hosted by the datalab contain the source of interest. 

In addition to MOCs, this notebook will also generate and display HiPs catlogs. While MOCs visualize survey footprints, HiPS will identify individual sources and provide information about each source when you place your curser over their points on Aladin. 


<a class="anchor" id="disclaimer"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., <a href="http://dx.doi.org/10.1117/12.2057445">"The NOAO Data Laboratory: a conceptual overview"</a>, SPIE, 9149, 2014

* <a href="https://datalab.noirlab.edu/disclaimers.php">Data Lab disclaimer</a>

* DRAGONS publication: Labrie et al., <a href="https://ui.adsabs.harvard.edu/abs/2019ASPC..523..321L/abstract">"DRAGONS - Data Reduction for Astronomy from Gemini Observatory North and South"</a>, ASPC, 523, 321L 

* <a href="https://zenodo.org/record/7776065#.ZDg5qOzMLUI">DRAGONS open source software publication</a>

<a class="anchor" id="imports"></a>
# Importing Python libraries

In [ ]:
from ipyaladin import Aladin

from mocpy import MOC

import astropy.units as u
from astropy.visualization.wcsaxes.frame import EllipticalFrame
from astropy.wcs import WCS
from astropy.table import Table

import matplotlib.pyplot as plt

<a class="anchor" id="Create_MOCs"></a>
# Create the MOCs

__NOTE:__ I think I should create another section called 'Accessing the data' and move 'Create the MOCs' further down.

To create the MOC that we will ultimately overlay onto Aladin, we are going to use the mocpy library. 

The first thing we will need to do is obtain the data from the Astro Data Lab. For this example, we will be working with data from three different surveys; The **DE**Cam **L**ocal **V**olume **E**xploration Survey [**DELVE**](https://datalab.noirlab.edu/delve/index.php), The **S**urvey of the **MA**gellanic **S**tellar **H**istory [**SMASH**](https://datalab.noirlab.edu/smash/smash.php), and The **D**ark **E**negery **S**urvey [**DES**](https://datalab.noirlab.edu/des/index.php).

To accesss the data, you need to access the Query Interface which is listed under Tools on the [Astro Data Lab](https://datalab.noirlab.edu/index.php) home page. Before submitting your query, you will need to log into your account or create a new one. Once logged in, all of the available datasets will be listed as bullets points on the left hand side of the screen. 

The data used in this example will come from the following catalogs:
- delve_dr2.objects 
- des_dr2.main
- smash_dr2.object

On the Query Interface, we are going to enter a command that follows this structure:

---
SELECT TOP 10000 * 

FROM **table**

WHERE random_id BETWEEN 25.0 AND 25.01

---

You will need to run this query three times, one for each of the tables listed above. All you need to do, is replace **table** in the example quey with the catalog name then click the blue 'Process' button. 

The first line of the query tells the interface to select all of the columns belonging to the top 10,000 rows that meet the criteria listed below. 

The second line tells the interface from which table to apply the criteria.

Finally, the third line utilizes the random_id that is assigned to each item in the table. The random values are all in the range 0.0. to 100.0. To acheive a good foot print from each table, we impose the condition that the random_id for each source should fall between 25.0 and 25.01. Although this range seems small, the catalogs that we are working with contain billions of sources which means this range will be sufficient. 

The image below shows the query that will obtain data from the DELVE survey. 

![DataLabQuery](DataLabQuery.png)